In [7]:
import json
import math

In [8]:
qasm = '''

OPENQASM 2.0;
include "hqslib1.inc";
qreg q0[6];
qreg q1[6];


creg meas0[6];
creg meas1[6];


creg hcheck[2];
creg hcheck1[2];

creg scratch[6];

creg twirl[2];


twirl = 0;

// Create first magic state
ry(pi/4) q0[0];
h q0[1];
sdg q0[1];


h q0[2];
h q0[4];
cx q0[2], q0[3];
cx q0[4], q0[5];
cx q0[2], q0[0];
cx q0[3], q0[1];
cx q0[0], q0[4];
cx q0[1], q0[5];
cx q0[4], q0[2];
cx q0[5], q0[3];







// Create second magic state
ry(pi/4) q1[0];
h q1[1];
sdg q1[1];

h q1[2];
h q1[4];
cx q1[2], q1[3];
cx q1[4], q1[5];
cx q1[2], q1[0];
cx q1[3], q1[1];
cx q1[0], q1[4];
cx q1[1], q1[5];
cx q1[4], q1[2];
cx q1[5], q1[3];



// Twirl here
if(twirl[0] == 1)h q0;
if(twirl[0] == 1)x q0[1];
if(twirl[0] == 1)x q0[3];
if(twirl[0] == 1)x q0[5];


// Twirl here
if(twirl[1] == 1)h q1;
if(twirl[1] == 1)x q1[1];
if(twirl[1] == 1)x q1[3];
if(twirl[1] == 1)x q1[5];



cy q1, q0;
sdg q1;
h q1;
measure q1 -> meas1;
scratch[0] = meas1[0]^meas1[2];
scratch[0] = scratch[0]^meas1[4];

if(scratch[0] == 0) ry(-pi/2) q0;
measure q0 -> meas0;


'''
qasms = []
for j in range(0,4):
    qasms.append(qasm.replace('twirl = 0;', 'twirl = %d;'%j))

In [9]:
with open("NFTMagicStateProtcol.json", "r") as file:
    jobs = json.load(file)

In [10]:
def checks(r,i):
    meas0 = r['meas0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False

    meas0 = r['meas1'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    return True

def success(r,i):
    meas0 = r['meas0'][i]
    return (int(meas0[5]) + int(meas0[3]) + int(meas0[1]))%2 == 0

In [11]:
fails = 0
totals = 0
shots = 0
for j in jobs:
    r = j['results']
    for i in range(0, len(r['meas0'])):
        shots += 1
        if (checks(r,i)):
            totals += 1
            if (not success(r,i)):
                fails += 1
print(fails/totals)
print(totals/shots)
import math
print(math.sqrt(fails)/totals)
print(fails, totals, shots)

0.0013199142055766375
0.90915
0.00019051320547422066
48 36366 40000


For the percentage of the time that the logical magic states have a non-trivial syndrome, we use the estimate from the fault-tolerant benchmarking experiment, in which all syndromes were extracted

In [12]:
s = 1 - 0.926125

In [13]:
p = fails/totals
n = totals
wilson = (1/(1 + 1/n))*(1/(2*n))*(math.sqrt(4*n*p*(1-p) + 1))
wilson_mean = 1/(1+ (1/n))*(p + (1/(2*n)))
print('protocol failure rate confidence interval: (',  wilson_mean - wilson, ',',  wilson_mean + wilson, ')',)

protocol failure rate confidence interval: ( 0.0011427486446361102 , 0.0015245046344355753 )


In [14]:
magic_state_fidelity = p/(2*(1-s))
print('magic state infidelity:', magic_state_fidelity)
print('magic state infidelity confidence interval: (', (wilson_mean - wilson)/(2*(1-s)), ',',  (wilson_mean + wilson)/(2*(1-s)), ')',)

magic state infidelity: 0.0007126004619120731
magic state infidelity confidence interval: ( 0.000616951623504446 , 0.0008230555456528953 )
